In [7]:
import pandas as pd
import numpy as np
import re

In [6]:
# Open the text file
with open('Data/BONUS_HASAM_202206235.txt', 'r') as f:
    lines_35 = f.readlines()

with open('Data/BONUS_HASAM_202206233.txt', 'r') as f:
    lines_33 = f.readlines()

lines = lines_35 + lines_33


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 9: invalid continuation byte

In [21]:
## Data Structure
## id = 9, fullname = 32, birth_date = 8, year = 4, zihui = 2, sug_isuk = 10, oved_dummy = 12,
## wage = 10, employer_name = 32, tik_nikui = 9, sug_divuh = 1, anaf_calcali = 4

# Create an empty list to store the data
data = []

# Iterate over the lines of the file
for line in lines:
    # Extract the first 10 characters and assign to id
    id = line[:9]
    full_name = line[9:41][::-1]
    birth_date = line[41:49]
    year = line[49:53]
    zihui = line[53:54]
    sug_isuk = line[54:64][::-1]
    oved_dummy = line[64:76][::-1]
    wage = line[76:86]
    employer_name = line[86:118][::-1]
    tik_nikui = line[118:127]
    sug_divuh = line[127:128]

    ## Append the data to the data list
    data.append((id, full_name, birth_date,
                 year, zihui, sug_isuk, oved_dummy,
                 wage, employer_name,
                 tik_nikui, sug_divuh))

# Create a data frame from the data list
df = pd.DataFrame(data, columns=['id', 'full_name', 'birth_date', 'year', 'zihui', 'sug_isuk', 'oved_dummy', 'wage', 'employer_name', 'tik_nikui', 'sug_divuh'])


In [22]:
## Filter out strings with non numeric characters in the wage column like "000000006K"

def filter_non_numeric_rows(column):
    return [val for val in column if bool(re.search(r'[^\d]', val))]
    
alphabet_list = list(filter_non_numeric_rows(df['wage']))

df = df[~df['wage'].isin(alphabet_list)]

In [23]:
## Data Types

## dtypes dict
dtypes = {'id': str,
          'full_name': str,
          'zihui':object,
          'sug_isuk': object,
          'oved_dummy': str,
          'wage': float,
          'employer_name': str,
          'tik_nikui': int,
          'sug_divuh': object,
          }

df = df.astype(dtypes)

## year to datetime
#df["year"] = pd.to_datetime(df["year"], format='%Y')

In [ ]:
df["wage"].describe()

In [24]:
# Duumies for employment months

## Replace כ with 1
df["oved_dummy"]= df["oved_dummy"].str.replace('כ', '1')
df["oved_dummy"]= df["oved_dummy"].str.replace(' ', '0')


month_cols = ["drop1","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12","drop2"]

## Split every character in df["oved_dummie"] to a new column
df[month_cols] = df["oved_dummy"].str.split("",expand=True)

## Drop the first and last columns
df = df.drop(columns=["drop1","drop2"])
## Drop from month_cols
month_cols = month_cols[1:-1]

for col in month_cols:
    ## Replace "" with NaN
    df[col] = df[col].replace('', 0)
    df[col] = df[col].astype(float)

In [25]:
## Drop non identified id's
print(df["zihui"].value_counts(normalize=True))
df = df.loc[(df["zihui"] != "ל")]

כ    1.0
Name: zihui, dtype: float64


In [26]:
## Add a variable for mean monthly wage
month_cols = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]
df["month_worked"] = df[month_cols].sum(axis=1)
df["mean_wage"] = (df["wage"]/df["month_worked"]).round(2)

for col in month_cols:
    df[col] = df[col]*df["mean_wage"]

    ## Group by year
df = df.groupby(["id","year"]).agg({"wage":"sum","1": "sum","2":"sum","3":"sum","4":"sum","5":"sum","6":"sum","7":"sum","8":"sum","9":"sum","10":"sum","11":"sum","12":"sum"}).reset_index()


In [27]:
## Reshape the data to long format (every month is a row for each id)
df = df.melt(id_vars=["id","year"], 
             value_vars=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"], 
             var_name="month", 
             value_name="wage_per_month")

In [ ]:
## To datetime format
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'], format='%Y-%m')
df = df.drop(columns=["month","year"])

In [ ]:
## save To CSV format
df.to_csv("Output/BONUS_HASAM_202206233_35.csv")